In [1]:
! pip install transformers sentencepiece -q

     |████████████████████████████████| 2.9 MB 32.0 MB/s 
     |████████████████████████████████| 1.2 MB 42.3 MB/s 
     |████████████████████████████████| 895 kB 49.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
     |████████████████████████████████| 596 kB 63.7 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 


In [2]:
! pip install -q gradio

     |████████████████████████████████| 2.0 MB 33.8 MB/s 
     |████████████████████████████████| 206 kB 53.2 MB/s 
     |████████████████████████████████| 1.9 MB 38.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 3.5 MB 42.7 MB/s 
     |████████████████████████████████| 961 kB 50.1 MB/s 


In [4]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)# Diverse Beam search



def generate_text(inp):
    context = inp
    text = "paraphrase: "+context + " </s>"
    encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    model.eval()
    diverse_beam_outputs = model.generate(
        input_ids=input_ids,attention_mask=attention_mask,
        max_length=128,
        early_stopping=True,
        num_beams=5,
        num_beam_groups = 5,
        num_return_sequences=5,
        diversity_penalty = 0.70)
    
    sent = tokenizer.decode(diverse_beam_outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    return sent
        

output_text = gr.outputs.Textbox()
gr.Interface(generate_text,"textbox", output_text).launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://36643.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://36643.gradio.app')